In [4]:
%matplotlib inline

import theano
import cPickle
import numpy as np
from theano import tensor, function
from pylab import imshow
from pylab import rcParams

from agnez.weight import grid2d
from fuel.datasets import MNIST
from fuel.streams import DataStream
from fuel.transformers import Mapping, Flatten
from fuel.schemes import SequentialScheme
from blocks.bricks.cost import MisclassificationRate
from rnn import allrotations, _meanize

#### Load models and data

In [8]:
batch_size = 128
n_steps = 20
mnist_train = MNIST("train", subset=slice(0, 50000))
mnist_valid = MNIST("train", subset=slice(50000, 60000))
mnist_test = MNIST("test")
trainstream = Mapping(Flatten(DataStream(mnist_train,
                          iteration_scheme=SequentialScheme(50000, batch_size))),
                          _meanize(n_steps))
validstream = Mapping(Flatten(DataStream(mnist_valid,
                                             iteration_scheme=SequentialScheme(10000,
                                                                               batch_size))),
                          _meanize(n_steps))
teststream = Mapping(Flatten(DataStream(mnist_test,
                                            iteration_scheme=SequentialScheme(10000,
                                                                              batch_size))),
                         _meanize(n_steps))

In [15]:
model_name = 'best_mnist.pkl'
i2h, h2o, rec1 = cPickle.load(file(model_name, 'r'))

#### Visualize weights

In [ ]:
rcParams['figure.figsize'] = 5,5
W = i2h.linear_transformations[0].W.get_value()
I = grid2d(W.T[:100])
imshow(I, cmap='gray')
W.shape

#### Reconstructions

In [ ]:
EPS = tensor.tensor3('noise')
PRE = i2h.apply(EPS)
HH = rec1.apply(PRE)
ZZ = h2o.apply(HH)
F = function([EPS], ZZ)
eps = np.random.normal(0.,1,(1,1000)).astype('float32')
eps = eps[np.newaxis].repeat(20, axis=0)
zz = F(eps)
imshow(zz[0].reshape((28,28)),cmap='gray')

In [ ]:
X = tensor.tensor3('features')
Z0 = i2h.apply(X)
Z1 = rec1.apply(Z0)
Y = h2o.apply(Z1)
F = function([X], Y)
E = function([X], tensor.sqr(X-Y).sum())

total_error = 0
total_energy = 0
for x_val in teststream.get_epoch_iterator():
    total_error += E(x_val[0])
    total_energy += (x_val[0]**2).sum()

In [ ]:
from agnez.inputs import image_sequence
rcParams['figure.figsize'] = 10, 5
y = F(x_val[0])
rec = image_sequence(y[:,0,:], (28,28))
imshow(rec, cmap='gray')

print "SNR (DB) = %f" % (10*np.log10(total_error))

In [ ]:
rec = image_sequence(x_val[0][:,0,:], (28,28))
imshow(rec, cmap='gray')

#### 2D Embedding

In [ ]:
zt = np.zeros((6e4, 1e3))
labels_t = np.zeros(6e4)
G = theano.function([X], Z1[-1])
for i, b in enumerate(trainstream.get_epoch_iterator()):
    first = i*100
    last = (i+1)*100
    zt[first:last] = G(b[0])
    labels_t[first:last] = b[1][:,0]

In [ ]:
z = np.zeros((1e4, 1e3))
labels = np.zeros(1e4)
G = theano.function([X], Z1[-1])
for i, b in enumerate(teststream.get_epoch_iterator()):
    first = i*100
    last = (i+1)*100
    z[first:last] = G(b[0])
    labels[first:last] = b[1][:,0]

In [ ]:
from agnez import embedding2d, embedding2dplot
labels=labels.astype('int')
ebd, mtd = embedding2d(z)
_ = embedding2dplot(ebd, labels)

#### Classification

In [ ]:
# from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier as KNC
knc = KNC(100)
knc.fit(zt, labels_t).score(z, labels)

In [ ]:
TT = tensor.matrix('targets')
probs = clf.apply(Z)
clf_error_rate = MisclassificationRate().apply(TT.flatten(), probs)
CLF = theano.function([X,TT], clf_error_rate)

error_rate = 0
for x_val in teststream.get_epoch_iterator():
    error_rate += CLF(x_val[0], x_val[1])
    
print "total error rate = %f" % (error_rate/100)

In [ ]:
y.shape

In [ ]:
print y.mean()
print y.var()